# Simple TDE Filter

Search for several broad line emission features characteristic of TDE spectra and select object with all or several of those features. Examples:
1. H$\alpha$ emission.
1. Missing O III $\lambda5007$ line.
1. etc.

In [2]:
import os
from glob import glob      # This is used for "wildcard" searches of folders.

from abc import ABC

from astropy.table import Table

from desispec.io import read_spectra, write_spectra
from desispec.spectra import stack as specstack
from desispec.coaddition import coadd_cameras

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import scipy.signal
from scipy.ndimage import gaussian_filter1d
from scipy.stats import norm
from scipy.optimize import curve_fit

## TDE Filtering Code

In [3]:
# Search code implemented as a class.
def gauss2(x, b, a, x0, sigma):
    """Implementation of 1D Gaussian with arbitrary normalization.
    
    Parameters
    ----------
    x : ndarray
        List of independent values (ordinates) where we perform the fit.
    b : float
        Free parameter: constant offset.
    a : float
        Free parameter: normalization.
    x0 : float
        Location parameter of the Gaussian.
    sigma : float
        Scale parameter (width) of the Gaussian.
    
    Returns
    -------
    y : ndarray
        Gaussian function N(x|a,b,x0,sigma) evaluated for all x.
    """
    return b + (a * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2)))

class TDEFilter:
    
    def __init__(self):
        self.heii_line_range = [4670., 4700.]
        self.oiii_line_range = [4992., 5022.]
        self.halpha_line_range = [6540., 6570.]
        
    def get_peak_in_range(self, wave, flux, wrange, prom=0, width=1):
        # Pick out the desired wavelength subrange of the spectrum.
        wmin, wmax = wrange
#        i, j = 0, len(wave)   # TEMPORARY: DELETE AFTER TESTING!
        i = np.abs(wmin - wave).argmin()
        j = np.abs(wmax - wave).argmin()
        
        # Apply the scipy peak finder to the subarray.
        peaks, info = scipy.signal.find_peaks(flux[i:j], prominence=prom, width=width)

        if len(peaks) > 0:
            # Pick out the maximum flux in this range (eliminates noise/local minima).
            k = flux[peaks].argmax()
            peakidx = peaks[k]
            peakflux = flux[peakidx]
            peakwave = wave[peakidx]
            peakwidth = info['widths'][k]
            return peakidx, peakwave, peakwidth, peakflux

        return None
    
    def snip_data_near_peak(self, wave, flux, good_peak, good_width):
        # Extract a range of values in pixel space (2.2*peak finder width).
        peak_edge = int(np.floor(2.2*(good_width)))
        i = np.maximum(0, good_peak - peak_edge)
        j = np.minimum(len(wave), good_peak + peak_edge)
        
        return wave[i:j], flux[i:j]
    
    def get_line_features(self, wave, flux, wrange):
        # Call the peak finder for the spectrum.
        peakinfo = self.get_peak_in_range(wave, flux, wrange)
        if peakinfo is not None:
            # Extract a range around the peak.
            peakidx, peakwave, peakwidth, peakflux = peakinfo
            subwave, subflux = self.snip_data_near_peak(wave, flux, peakidx, peakwidth)
        
            # Fit the feature to a 1D Gaussian.
            pars, cov = curve_fit(gauss2, subwave, subflux, p0=[1., peakflux, peakwave, peakwidth])
            return pars
        return None
            
    def is_tde(self, wave, flux):
        
        # Call the peak finder for the spectral features we want.
        try:
            heii_pars = self.get_line_features(wave, flux, self.heii_line_range)
            oiii_pars = self.get_line_features(wave, flux, self.oiii_line_range)
            halpha_pars = self.get_line_features(wave, flux, self.halpha_line_range)
        except:
            return False
        
#         print(heii_pars)
#         print(oiii_pars)
#         print(halpha_pars)
            
        # Try to identify the TDE-like features.
        # Must have Ha and HeII lines.
        if heii_pars is None or halpha_pars is None:
            return False
        he_ha_ratio = heii_pars[1] / halpha_pars[1]
        
        # OIII line not required; set to zero by default.
        o_ha_ratio = 0.
        if oiii_pars is not None:
            o_ha_ratio = oiii_pars[1] / halpha_pars[1]

        # Condition: high HeII/Ha and little to no OIII/Ha.
        return he_ha_ratio > 0.9 and o_ha_ratio < 0.1

## Access to Tile ToO from July 2021

This is a dummy data set we'll use as a sandbox to test the TDE filter.

Locally cache the selected spectra so we don't have to have a time consuming loop and selectio over data.

In [9]:
basedir = os.environ['DESI_SPECTRO_REDUX']
redux = 'daily'
tiledata = 'tiles/cumulative'

tiles = ['80980', '80981']
tiledate = '20210708'

coadd_file = 'selected_coadds_80980_80981.fits'
use_cache = False

if os.path.exists(coadd_file) and use_cache:
    spectra = read_spectra(coadd_file)
else:    
    spectra = None
    # Loop over all tiles in our tile list.
    for tile in tiles:
        tilefolder = '{}/{}/{}/{}/{}'.format(basedir, redux, tiledata, tile, tiledate)
        coadds = sorted(glob('{}/coadd*.fits'.format(tilefolder)))

        # Loop over all petals in the spectroscopic reduction.
        for coadd in coadds:
            # Read the spectra and the best-fit redshifts from redrock and combine into one stack of spectra.
            spec = read_spectra(coadd)
            zbest_files = coadd.replace('coadd', 'zbest')
            zbest = Table.read(zbest_files, 'ZBEST')

            # Select spectra with good redshifts at low z to pick TDE candidates.
            select = (zbest['DELTACHI2']>25) & (zbest['ZWARN']==0) & ((zbest['SPECTYPE']=='GALAXY') | (zbest['SPECTYPE']=='QSO'))

            # Match TARGETID between good redshifts in zbest and the spectra fibermap table.
            idx = np.in1d(spec.fibermap['TARGETID'], zbest['TARGETID'][select])

            # Coadd our selected (good redshift) spectra across the b, r, z cameras.
            cspec = coadd_cameras(spec[idx])
            cspec.scores = None
            cspec.extra_catalog = zbest[select]

            # Accumulate spectra from all petals into one object, "spectra."
            if spectra is None:
                spectra = cspec
            else:
                spectra = specstack([spectra, cspec])

    write_spectra(coadd_file, spectra)
    zbest = spectra.extra_catalog

INFO:spectra.py:282:read_spectra: iotime 0.769 sec to read coadd-0-80980-thru20210708.fits at 2021-08-17T13:47:27.299143
INFO:spectra.py:282:read_spectra: iotime 0.782 sec to read coadd-1-80980-thru20210708.fits at 2021-08-17T13:47:36.261030
INFO:spectra.py:282:read_spectra: iotime 0.665 sec to read coadd-2-80980-thru20210708.fits at 2021-08-17T13:47:45.042027
INFO:spectra.py:282:read_spectra: iotime 0.605 sec to read coadd-3-80980-thru20210708.fits at 2021-08-17T13:47:53.285705
INFO:spectra.py:282:read_spectra: iotime 0.694 sec to read coadd-4-80980-thru20210708.fits at 2021-08-17T13:48:01.427237
INFO:spectra.py:282:read_spectra: iotime 0.823 sec to read coadd-5-80980-thru20210708.fits at 2021-08-17T13:48:10.902195
INFO:spectra.py:282:read_spectra: iotime 0.513 sec to read coadd-6-80980-thru20210708.fits at 2021-08-17T13:48:20.855642
INFO:spectra.py:282:read_spectra: iotime 0.714 sec to read coadd-7-80980-thru20210708.fits at 2021-08-17T13:48:30.672558
INFO:spectra.py:282:read_spectra

In [11]:
spectra.fibermap

TARGETID,PETAL_LOC,DEVICE_LOC,LOCATION,FIBER,FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,REF_EPOCH,LAMBDA_REF,FA_TARGET,FA_TYPE,OBJTYPE,PRIORITY,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKID,BRICK_OBJID,MORPHTYPE,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MASKBITS,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,BRICKNAME,EBV,FLUX_W1,FLUX_W2,FLUX_IVAR_W1,FLUX_IVAR_W2,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,SERSIC,SHAPE_R,SHAPE_E1,SHAPE_E2,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,DESI_TARGET,BGS_TARGET,MWS_TARGET,SCND_TARGET,PLATE_RA,PLATE_DEC,PSF_TO_FIBER_SPECFLUX,TILEID,COADD_NUMEXP,COADD_EXPTIME,MEAN_DELTA_X,RMS_DELTA_X,MEAN_DELTA_Y,RMS_DELTA_Y,MEAN_FIBER_X,MEAN_FIBER_Y,MEAN_FIBER_RA,MEAN_FIBER_DEC,MEAN_FIBERASSIGN_X,MEAN_FIBERASSIGN_Y,FIRST_NIGHT,LAST_NIGHT,NUM_NIGHT,FIRST_EXPID,LAST_EXPID,NUM_EXPID,FIRST_TILEID,LAST_TILEID,NUM_TILEID,FIRST_FIBER,LAST_FIBER,NUM_FIBER,FIRST_MJD,LAST_MJD,NUM_MJD
int64,int16,int32,int64,int32,int32,float64,float64,float32,float32,float32,float32,int64,uint8,bytes3,int32,float64,int32,int16,int32,int32,bytes4,float32,float32,float32,float32,float32,float32,int16,int64,bytes2,float32,float32,float32,float32,bytes8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes1,int64,int64,int64,int64,int64,int64,float64,float64,float64,int32,int16,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,int32,int32,int16,int32,int32,int16,int32,int32,int16,int32,int32,int16,float32,float32,int16
43978251332224297,0,522,522,478,0,218.813975,35.121953,0.0,0.0,2015.5,5400.0,4611686018427387904,1,TGT,950,0.9742937853685163,5,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0,0,,0.0,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,950,1,4611686018427387904,0,0,576460752303423488,218.813975,35.121953,0.7926560553510068,80980,2,2112.8784,-0.004,0.004,-0.004,0.004,37.89,-399.857,218.81395557359048,35.12196767708496,37.86777,-399.8724,20210708,20210708,1,97938,97939,2,80980,80980,1,478,478,1,59404.168,59404.184,2
43978251332224310,0,520,520,471,0,218.915517,35.095306,0.0,0.0,2015.5,5400.0,4611686018427387904,1,TGT,950,0.846384418152861,5,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0,0,,0.0,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,950,1,4611686018427387904,0,0,576460752303423488,218.915517,35.095306,0.7925311896943021,80980,2,2112.8784,0.0,0.0,-0.001,0.001,16.854,-407.059,218.91551698770067,35.09530962627788,16.834837,-407.08102,20210708,20210708,1,97938,97939,2,80980,80980,1,471,471,1,59404.168,59404.184,2
43978251336418634,0,503,503,497,0,219.080169,35.122149,0.0,0.0,2015.5,5400.0,4611686018427387904,1,TGT,950,0.34329178283455775,5,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0,0,,0.0,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,950,1,4611686018427387904,0,0,576460752303423488,219.080169,35.122149,0.7924976169140224,80980,2,2112.8784,-0.001,0.001,-0.001,0.001,-17.279,-399.621,219.08016420293768,35.122152639496015,-17.301447,-399.6295,20210708,20210708,1,97938,97939,2,80980,80980,1,497,497,1,59404.168,59404.184,2
43978251340612978,0,483,483,345,0,219.292823,35.10903,0.0,0.0,2015.5,5400.0,4611686018427387904,1,TGT,950,0.7245738764353887,5,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0,0,,0.0,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,950,1,4611686018427387904,0,0,576460752303423488,219.292823,35.10903,0.7926175156620128,80980,2,2112.8784,-0.004,0.004,0.001,0.001,-61.447,-403.418,219.29280369081238,35.10902625495025,-61.476753,-403.43625,20210708,20210708,1,97938,97939,2,80980,80980,1,345,345,1,59404.168,59404.184,2
43978256277309409,0,436,436,238,0,218.573868,35.330598,0.0,0.0,2015.5,5400.0,4611686018427387904,1,TGT,950,0.37643002599810205,5,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0,0,,0.0,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,950,1,4611686018427

In [16]:
spectra.extra_catalog[spectra.extra_catalog['SPECTYPE'] == 'QSO']

TARGETID,CHI2,COEFF [10],Z,ZERR,ZWARN,NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,NUMEXP,NUMTILE
int64,float64,float64,float64,float64,int64,int64,bytes6,bytes20,int64,float64,int32,int32
43978256281501745,13367.018755748868,0.003186404039672735 .. 0.0,2.24056182051903,0.0002633956991651117,0,7916,QSO,,4,6536.854853831232,0,1
43978261214003262,10079.876761939377,0.001989441964014777 .. 0.0,0.8705321741721923,0.0001771566952248695,0,7926,QSO,,4,484.48192527517676,0,1
43978261214005779,10624.27079334855,0.001538652402187461 .. 0.0,0.3160512950279755,2.547907193497382e-05,0,7916,QSO,,4,1593.7279356531799,0,1
43978261218197606,10197.292956903577,0.003957286377801545 .. 0.0,1.0558361781254213,0.00013536016530823585,0,7915,QSO,,4,1337.8472392261028,0,1
43978266138116199,11332.984649889171,0.004744578605564389 .. 0.0,0.737254183035968,9.586718830787771e-05,0,7927,QSO,,4,775.0246099308133,0,1
43978266125533208,12453.89845597744,0.006649859076654186 .. 0.0,0.9755788307581459,0.00011738479897894504,0,7920,QSO,,4,3455.0487359071085,0,1
43978271032868874,16222.728982925415,0.009965818716048534 .. 0.0,0.6712054201914494,1.9222047822204384e-05,0,7921,QSO,,4,10663.224858283997,0,1
43978275931820120,11424.597808623686,0.00024015913897280396 .. 0.0,0.26151659907812036,6.838980329251752e-05,0,7919,QSO,,4,31.150422936305404,0,1
616093743556068151,10032.598878026009,-4.179072109702796e-05 .. 0.0,1.7922520489511575,0.00021612551501063452,0,7924,QSO,,4,28.233668066561222,0,1
